# Views in Databricks
Views are logical tables created by defining a query on existing tables without physically storing data.
In this notebook, we’ll cover:
- Types of Views
- How to Create Views
- Managing Views
- Examples with Temporary and Global Temporary Views

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ViewsStudy').getOrCreate()

# This notebook assumes you have an active Spark session running in a Databricks environment.

### Sample Data
To understand views, we’ll create a sample `orders` table with `customer_id`, `order_id`, and `order_date`.

In [ ]:
orders_data = [
    (1, 101, '2024-10-01'),
    (2, 102, '2024-10-02'),
    (1, 103, '2024-10-03'),
    (3, 104, '2024-10-04'),
    (2, 105, '2024-10-05'),
]
columns = ['customer_id', 'order_id', 'order_date']

orders_df = spark.createDataFrame(data=orders_data, schema=columns)
orders_df.show()

# Register as a temporary table
orders_df.createOrReplaceTempView('orders')

### Creating a Temporary View
Temporary views are session-scoped and disappear once the session ends. Below, we’ll create a temporary view to summarize orders by `customer_id`.

In [ ]:
spark.sql('''
    CREATE OR REPLACE TEMP VIEW customer_orders AS
    SELECT customer_id, COUNT(order_id) AS total_orders
    FROM orders
    GROUP BY customer_id
''')

temp_view_df = spark.sql('SELECT * FROM customer_orders')
temp_view_df.show()

### Creating a Global Temporary View
Global temporary views are accessible across multiple sessions within the same cluster.
They are registered in the `global_temp` database.

In [ ]:
spark.sql('''
    CREATE OR REPLACE GLOBAL TEMP VIEW active_customers AS
    SELECT customer_id, MAX(order_date) AS last_order_date
    FROM orders
    GROUP BY customer_id
''')

global_temp_view_df = spark.sql('SELECT * FROM global_temp.active_customers')
global_temp_view_df.show()

### Managing Views
Views can be replaced or dropped as needed. We use `CREATE OR REPLACE` to update views and `DROP VIEW` to delete them.

In [ ]:
spark.sql('DROP VIEW IF EXISTS customer_orders')
spark.sql('DROP VIEW IF EXISTS global_temp.active_customers')

# Verifying views have been dropped
try:
    spark.sql('SELECT * FROM customer_orders').show()
except Exception as e:
    print('View customer_orders has been dropped.')

try:
    spark.sql('SELECT * FROM global_temp.active_customers').show()
except Exception as e:
    print('View global_temp.active_customers has been dropped.')

### Best Practices
- **Name Views Descriptively**: Use a naming convention that describes the view’s purpose.
- **Avoid Over-Nesting**: Nested views can become difficult to manage and impact performance.
- **Access Control**: Set appropriate permissions, especially for views with sensitive data.

## Summary
In this notebook, we covered:
- Types of views in Databricks (Temporary and Global Temporary Views)
- Creating, querying, and managing views
- Best practices for using views

Views are powerful tools for abstracting complex queries, improving code readability, and enhancing data security.